In [46]:
import pandas as pd
import numpy as np
from json import dumps
import pandana as pdna
import Queue
import time

In [2]:
census_block_db = pd.read_csv('./data/net_am_peak/block_am_closest_node.csv', dtype={'GEOID10':str})

In [3]:
store = pd.HDFStore('./data/net_am_peak/final_net.h5', "r")
nodes = store.nodes
edges = store.edges

In [ ]:
net=pdna.Network(nodes["x"], nodes["y"], edges["from"], edges["to"],edges[["weight"]])

In [4]:
reach_dict = {}
dist = 10
nodes_num = len(census_block_db)
name ='am20'

In [5]:
net.set_pois(name , dist, nodes_num, census_block_db['INTPTLON10'], census_block_db['INTPTLAT10'])

In [6]:
reach_pd = net.nearest_pois(distance=dist,category=name, num_pois=10, include_poi_ids=True)

/Users/huangziqiang/IMSCProject/env/lib/python2.7/site-packages/pandana/network.py:576: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df2[col][s == -1] = np.nan


In [8]:
reach_pd2 = reach_pd.iloc[:,10:20]

In [11]:
save_path = './data/nearest/'
for i in range(0, len(census_block_db)):
    row = census_block_db.iloc[i]
    file_name = save_path + row['GEOID10'] + ".json"
    node_id = row['node_id']
    nearest = reach_pd2.loc[node_id].dropna().astype(int)
    n_list = []
    for n_id in nearest:
        n_list.append(census_block_db.iloc[n_id, 1])
    with open(file_name, "w") as geojson:
        geojson.write(dumps({"nearest": n_list}, indent=2) + "\n")

In [ ]:
#second method below

In [42]:
net_dict = {}
for i in range(0, len(edges)):
    edge = edges.iloc[i]
    if net_dict.has_key(int(edge['from'])):
        net_dict.get(int(edge['from'])).append((int(edge['to']), edge['weight']))
    else:
        net_dict[int(edge['from'])] = [(int(edge['to']),edge['weight'])]

In [43]:
def bfs(start, distance):
    queue = Queue.Queue()
    visited = set()
    queue.put((start,0))
    visited.add(start)
    while not queue.empty():
        cur = queue.get()
        next_nodes = net_dict.get(cur[0])
        if next_nodes is None:
            continue
        for node in next_nodes:
            dist = cur[1] + node[1]
            if node[0] in visited or dist > distance:
                continue
            queue.put((node[0], dist))
            visited.add(node[0])
    return visited

In [63]:
id_dict = {}
for i in range(0, len(census_block_db)):
    row = census_block_db.iloc[i]
    node_id = row['node_id']
    geo_id = row['GEOID10']
    if id_dict.has_key(node_id):
        id_dict.get(node_id).append(geo_id)
    else:
        id_dict[node_id] = [geo_id]

In [67]:
distance = 20
save_path = './data/nearest' + str(distance) + '/'
start = time.time()
for i in range(0, len(census_block_db)):
    row = census_block_db.iloc[i]
    file_name = save_path + row['GEOID10'] + ".json"
    node_id = row['node_id']
    nearest = bfs(node_id, distance)
    n_list = []
    for n_id in nearest:
        if id_dict.has_key(n_id):
            n_list = n_list + id_dict[n_id]
    with open(file_name, "w") as geojson:
        geojson.write(dumps({"nearest": n_list}, indent=2) + "\n")
end = time.time()
print end - start

860.913890839


In [47]:
start = time.time()
nearest = bfs(1, 20)
end = time.time()
print end - start

0.0109589099884


In [64]:
id_dict

{131072: ['060371894001020'],
 305835: ['060375514012011', '060375514012010'],
 131076: ['060373017022006'],
 131078: ['060375545223012'],
 131080: ['060374028011001', '060374028011003'],
 393226: ['060374020021020'],
 174581: ['060377016014004'],
 14: ['060379800241037'],
 218456: ['060372702003014', '060372702003013'],
 131091: ['060371152012004'],
 131092: ['060371172022002'],
 131093: ['060371153011006'],
 131095: ['060371172012007'],
 393240: ['060374027062000'],
 288059: ['060372655203010'],
 262170: ['060375730021000'],
 131100: ['060371133031016'],
 131101: ['060371133032020'],
 262174: ['060375750023002'],
 327685: ['060379800331130', '060379800331131'],
 131106: ['060371153011015'],
 349531: ['060371345201022'],
 36: ['060373111003013'],
 131112: ['060371132371009'],
 41: ['060371242012011'],
 196615: ['060376021041017', '060376021041026'],
 131116: ['060375745001002'],
 131117: ['060375318003008'],
 131118: ['060375318003011'],
 262191: ['060375774001006'],
 131121: ['060378

In [61]:
len(nearest)

976

In [14]:
census_block_db

,Unnamed: 0,GEOID10,INTPTLAT10,INTPTLON10,node_id,net_lat,net_lon
0,0,060373006004026,34.197078,-118.231718,105686,34.197488,-118.230563
1,1,060373106012020,34.191093,-118.326216,385060,34.191174,-118.325751
2,2,060373102023003,34.190033,-118.295874,19961,34.191326,-118.294945
3,3,060373007015000,34.202718,-118.237676,351970,34.204203,-118.237912
4,4,060373007015006,34.191512,-118.230884,53415,34.190716,-118.229702
5,5,060371278052004,34.191199,-118.461271,87534,34.190304,-118.460847
6,6,060371277122002,34.195727,-118.471103,127162,34.195322,-118.470555
7,7,060371279202007,34.196203,-118.437260,61711,34.194844,-118.437812
8,8,060371310202017,34.196664,-118.537958,36230,34.195683,-118.537324
9,9,060373106022002,34.196748,-118.320611,24771,34.195841,-118.320774


In [49]:
nodes

,x,y
id_int,,
1,-118.541748,34.172424
2,-118.542587,34.171299
3,-118.538841,34.169781
4,-118.535866,34.169090
5,-118.535851,34.172226
6,-118.530487,34.172241
7,-118.526970,34.172161
8,-118.521706,34.172142
9,-118.518196,34.172180
